# 加速：非同步爬蟲

* 了解非同步爬蟲加速原理與實作

## 作業目標

* 比較一下非同步爬蟲跟多線程爬蟲的差異是什麼？各自的優缺點為何？

[事件驅動](https://ifun01.com/NAIKFIC.html)


[異步編程與多線程的區別](https://www.itread01.com/content/1553169850.html)

![alt text](https://img.ifun01.com/images/2016/10/12/100723_uEHSSU.jpg!r550x0.jpg)

In [1]:
# 正常情況的爬蟲所需時間

##  優點：順序執行。
##  缺點：我不參與但是我要幹等著，效率低下。

import requests
import time

urls = ['https://www.ettoday.net/news/news-list.htm'] * 5

startTime = time.time()

for url in urls:
    r = requests.get(url)
    # url = r.url
    # print(url)

finishTime = time.time()
print("single thread:", finishTime - startTime)  # 正常情況的爬蟲所需時間

single thread: 6.810043573379517


In [2]:
# 多線程(執行緒)  

##  優點：使用平行處理，在同一個時間可以執行多項任務，而達到多工的效果。
##  缺點：只能在單個CPU Core下執行，線程間的共享變量可能造成死鎖的出現。

import requests
import _thread
import time

urls = ['https://www.ettoday.net/news/news-list.htm'] * 5

startTime = time.time()

for url in urls:
    _thread.start_new_thread( requests.get, (url, ) )

finishTime = time.time()
print("multiple thread:", finishTime - startTime) # 利用 thread 的爬蟲所需時間


multiple thread: 0.00037097930908203125


In [3]:
# 非同步

##  優點：可以不循序的執行程式，有效的利用系統資源。減少共享變量的數量 (減少了死鎖的可能)。
##  缺點：每個請求在執行的過程中，主要使用回調方式進行處理，可能要消耗大量的系統資源。

# !pip install aiohttp
# !pip install nest_asyncio
import aiohttp, asyncio
import nest_asyncio
nest_asyncio.apply()

url = 'https://www.ettoday.net/news/news-list.htm'

async def job(session):
    response = await session.get(url)                               #等待並切換
    return str(response.url)

async def main(loop):
    async with aiohttp.ClientSession() as session:                  #官網推薦建立Session的形式,也可以直接用request
        tasks = [loop.create_task(job(session)) for _ in range(5)]
        finished, unfinished = await asyncio.wait(tasks)            #收集完成的結果,會返回完成的和沒完成的,等全部都完成了才返回
        # all_results = [r.result() for r in finished]              #獲取所有結果
        # print(all_results)

t1 = time.time()
loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))
loop.close()
print("Async total time:", time.time() - t1)  # 利用非同步爬蟲所需時間

Async total time: 1.1076490879058838
